### Feature Engineering (ORDERBOOK 부문)

In [ ]:
# df_orderbook_pd = df_orderbook.compute() # dask dataframe => pandas dataframe

# for testing:
df_orderbook_pd2 = df_orderbook_pd.iloc[:]

df_orderbook_pd2['datetime'] = pd.to_datetime(df_orderbook_pd2['datetime'])
# datetime을 1분 단위로 정리하는 변수 (dt_1m)을 만듬 (이는 데이터를 '정리'하는 데에 활용될 단위임)
df_orderbook_pd2['dt_1m'] = df_orderbook_pd2['datetime'].dt.floor('1min').dt.strftime('%Y-%m-%d %H:%M:%S')
df_orderbook_pd2['dt_1m'] = pd.to_datetime(df_orderbook_pd2['dt_1m'])
df_orderbook_pd2.set_index('dt_1m', inplace=True, drop = False)

# Define the size of the moving window (10 minutes in this case)
window_size = pd.Timedelta(minutes=10)

# Create an empty list to store features for each rolling window
liq_last_1 = []
liq_last_2 = []
liq_last_5 = []
liq_last_10 = []
liq_last_15 = []
bidask_spread_0 = []
bidask_spread_1 = []

temp_index = df_orderbook_pd2.index.unique().tolist()
# Iterate through each time window using the rolling method
for window_start in temp_index[:-10]:
    window_end = window_start + window_size

    # print("window_start:", window_start, " & window_end:", window_end)

    # Filter the data for each rolling window
    rolling_window_data = df_orderbook_pd2.loc[(df_orderbook_pd2.index >= window_start) & (df_orderbook_pd2.index < window_end)]
    # previous_price = rolling_window_data['ending_price_b1m'].iloc[0] # 10분 bin이 시작하기 직전 마지막 trade price 값을 의미.
    # rolling_window_data['log_return_from_pp'] = np.log(rolling_window_data['trade_price'] / previous_price)

    # VARIABLE: liquidity measures
    last_order_book = rolling_window_data.iloc[-1]
    wap_1 = (last_order_book['orderbook_bp_0'] * last_order_book['orderbook_as_0'] + last_order_book['orderbook_ap_0'] * last_order_book['orderbook_bs_0'])/(last_order_book['orderbook_bs_0'] + last_order_book['orderbook_as_0'])
    
    liq_1 = 0
    for i in range(1):
        liq_1 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

    liq_2 = 0
    for i in range(2):
        liq_2 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

    liq_5 = 0
    for i in range(5):
        liq_5 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

    liq_10 = 0
    for i in range(10):
        liq_10 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

    liq_15 = 0
    for i in range(15):
        liq_15 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

    liq_last_1.append(liq_1)
    liq_last_2.append(liq_2)
    liq_last_5.append(liq_5)
    liq_last_10.append(liq_10)
    liq_last_15.append(liq_15)

    # VARIABLE: bidask spread
    ob_price_diff = []
    for i in range(14):
        ob_price_diff.append(last_order_book[f'orderbook_bp_{i}'] - last_order_book[f'orderbook_bp_{i+1}'])
        ob_price_diff.append(last_order_book[f'orderbook_ap_{i+1}'] - last_order_book[f'orderbook_ap_{i}'])
    tick_size = min(ob_price_diff)

    # 마지막 1분 정보만 추출해서 bid ask spread의 평균을 구하자 (마지막 타이밍의 orderbook만 쓰는 것보다 낫지 않을까?)
    last_1m = rolling_window_data.loc[(rolling_window_data.dt_1m == rolling_window_data['dt_1m'].iloc[-1])]

    ba_sp_0 = (last_1m['orderbook_ap_0'] - last_1m['orderbook_bp_0'])/tick_size
    bidask_spread_0.append(ba_sp_0.mean())
    ba_sp_1 = (last_1m['orderbook_ap_1'] - last_1m['orderbook_bp_1'])/tick_size
    bidask_spread_1.append(ba_sp_1.mean())
    
    # liq_1 = [last_order_book['orderbook_bs_0']/(wap_1 - last_order_book['orderbook_bp_0']) + last_order_book['orderbook_as_0']/(last_order_book['orderbook_ap_0'] - wap_1)]
    # liq_1 = liq_1 +  

# Create a new DataFrame to store the results
result_df_orderbook = pd.DataFrame({
    'window_start': temp_index[:-10],
    'window_end': temp_index[10:],
    'liq_last_1': liq_last_1,
    'liq_last_2': liq_last_2,
    'liq_last_5': liq_last_5,
    'liq_last_10': liq_last_10,
    'liq_last_15': liq_last_15,
    'bidask_spread_0': bidask_spread_0,
    'bidask_spread_1': bidask_spread_1
})

# Reset the index of the result DataFrame
result_df_orderbook.reset_index(drop=True, inplace=True)

# time_id 만들어주기.
result_df_orderbook['window_start'] = pd.to_datetime(result_df_orderbook['window_start'])
result_df_orderbook['time_id'] = result_df_orderbook['window_start'].dt.floor('1min').dt.strftime('%Y-%m-%d %H:%M:%S')
result_df_orderbook['time_id'] = pd.to_datetime(result_df_orderbook['time_id'])

###### To-do : 450/300/150초 기준 변수 만들기.

time_intervals = [150, 300, 450]

# Loop through each time interval and create new columns
for interval in time_intervals:
    # Create new time intervals (window_start and window_end)
    result_df_orderbook[f'window_start_{interval}'] = result_df_orderbook['window_start'] + pd.Timedelta(seconds=interval)
    result_df_orderbook[f'window_end_{interval}'] = result_df_orderbook['window_end'] + pd.Timedelta(seconds=interval)

    result_df_orderbook[f'liq_last_1_{interval}'] = result_df_orderbook[f'liq_last_1']
    result_df_orderbook[f'liq_last_2_{interval}'] = result_df_orderbook[f'liq_last_2']
    result_df_orderbook[f'liq_last_5_{interval}'] = result_df_orderbook[f'liq_last_5']
    result_df_orderbook[f'liq_last_10_{interval}'] = result_df_orderbook[f'liq_last_10']
    result_df_orderbook[f'liq_last_15_{interval}'] = result_df_orderbook[f'liq_last_15']

    result_df_orderbook[f'bidask_spread_0_{interval}'] = result_df_orderbook[f'bidask_spread_0']
    result_df_orderbook[f'bidask_spread_1_{interval}'] = result_df_orderbook[f'bidask_spread_1']

# 첫 1분 및 마지막 11분은 데이터가 complete 하지 않을 것이므로 삭제.
start_time = result_df_orderbook['time_id'].iloc[0]
end_time = result_df_orderbook['time_id'].iloc[-11]
result_df_orderbook_flt = result_df_orderbook[(result_df_orderbook['time_id'] > start_time) & (result_df_orderbook['time_id'] < end_time)]

## result_df & result_df_orderbook 두개를 concat 하자.
result_df_orderbook_flt = result_df_orderbook_flt.drop(columns = ['window_end', 'time_id'])
combined_result_df = pd.merge(result_df_flt, result_df_orderbook_flt, on='window_start', suffixes=('_ticker', '_orderbook'))


